# Tensorflow basic
----------------------------
## Tensorflow.Saver
------------------------
[Tensorflow saver tutorial](https://www.tensorflow.org/programmers_guide/variables)  

In [1]:
import tensorflow as tf
import os

In [2]:
# Create two variables.
a = tf.Variable(20)
b = tf.Variable(3)
add = a.assign_add(b)

In [3]:
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

In [4]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [5]:
if not os.path.exists('ckpt'):
    os.makedirs('ckpt')
saver.save(sess, 'ckpt/first.ckpt')

'ckpt/first.ckpt'

In [6]:
print('a: ', a.eval())

a:  20


In [7]:
print(add.eval())

23


In [8]:
print('a: ', a.eval())

a:  23


In [9]:
saver.save(sess, 'ckpt/second.ckpt')

'ckpt/second.ckpt'

In [10]:
saver.restore(sess, 'ckpt/first.ckpt')
print(a.eval())

INFO:tensorflow:Restoring parameters from ckpt/first.ckpt
20


In [11]:
saver.restore(sess, 'ckpt/second.ckpt')
print(a.eval())

INFO:tensorflow:Restoring parameters from ckpt/second.ckpt
23


## Variable scope, name scope
----------------------
[Tensorflow sharing variables](https://www.tensorflow.org/programmers_guide/variable_scope)  
[StackOverflow: What's the difference of name scope and a variable scope in tensorflow?](https://stackoverflow.com/questions/35919020/whats-the-difference-of-name-scope-and-a-variable-scope-in-tensorflow)

Let's begin by a short introduction to variable sharing. It is a mechanism in TensorFlow that allows for sharing variables accessed in different parts of the code without passing references to the variable around. 

The method [tf.get_variable](https://www.tensorflow.org/api_docs/python/tf/get_variable) can be used with the name of the variable as argument to either **create a new variable** with such name or **retrieve the one** that was created before. 

This is different from using the [tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable) constructor which will **create a new variable every time** it is called (and potentially add a suffix to the variable name if a variable with such name already exists). It is for the purpose of the variable sharing mechanism that a separate type of scope (variable scope) was introduced.



In [12]:
print(a, a.name)
print(b, b.name)
print(add, add.name)

<tf.Variable 'Variable:0' shape=() dtype=int32_ref> Variable:0
<tf.Variable 'Variable_1:0' shape=() dtype=int32_ref> Variable_1:0
Tensor("AssignAdd:0", shape=(), dtype=int32_ref) AssignAdd:0


In [13]:
with tf.name_scope("my_scope"):
    v1 = tf.get_variable("v1", [1], dtype=tf.float32) # 'name scope' will be ignored.
    v2 = tf.Variable(1, name="v2", dtype=tf.float32)
    add_name_scope = tf.add(v1, v2)

In [14]:
print(v1.name)  # var1:0add_name_scope
print(v2.name)  # my_scope/var2:0
print(add_name_scope.name)   # my_scope/Add:0

v1:0
my_scope/v2:0
my_scope/Add:0


In [15]:
with tf.variable_scope("my_scope2"):
    v3 = tf.get_variable("v3", [1], dtype=tf.float32)
    v4 = tf.Variable(1, name="v4", dtype=tf.float32)
    add_variable_scope = tf.add(v3, v4)

In [16]:
print(v3.name)
print(v4.name)
print(add_variable_scope.name)

my_scope2/v3:0
my_scope2/v4:0
my_scope2/Add:0


In [17]:
with tf.name_scope("foo"):
    with tf.variable_scope("var_scope"):
        v5 = tf.get_variable("var_share", [1])
        square = tf.square(v5)
with tf.name_scope("bar"):
    with tf.variable_scope("var_scope", reuse=True): # sharing variable
        v6 = tf.get_variable("var_share", [1])
        sqrt = tf.sqrt(v6)

In [18]:
assert v5 == v6 #if not condition: raise AssertionError(), 'assert' only works on 'Debug' mode
print(v5.name)  # var_scope/var:0
print(v6.name)  # var_scope/var:0
print(square.name) # foo/var_scope/Square:0
print(sqrt.name) # bar/var_scope/Sqrt:0

var_scope/var_share:0
var_scope/var_share:0
foo/var_scope/Square:0
bar/var_scope/Sqrt:0


In [21]:
# code from 'https://stackoverflow.com/questions/38189119/simple-way-to-visualize-a-tensorflow-graph-in-jupyter'
import numpy as np
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add()
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [22]:
show_graph(tf.get_default_graph())